In [4]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# Función para contar los dedos levantados
def count_fingers(hand_landmarks):
    finger_tips = [8, 12, 16, 20]  # Índices de los puntos clave de las puntas de los dedos
    finger_foldings = [6, 10, 14, 18]  # Índices de las articulaciones cercanas a la palma

    fingers_up = 0
    for tip, folding in zip(finger_tips, finger_foldings):
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[folding].y:
            fingers_up += 1
    if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
        fingers_up += 1

    return fingers_up

# Captura de video desde la webcam
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        fingers_count = 0

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

                fingers_count = count_fingers(hand_landmarks)

        # Voltear la imagen en modo selfie
        flipped_image = cv2.flip(image, 1)

        # Agregar el texto después de voltear
        cv2.putText(
            flipped_image, f"Dedos levantados: {fingers_count}", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Mostrar la imagen con el texto correcto
        cv2.imshow('MediaPipe Hands', flipped_image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()
